In [10]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi 
import json
from pathlib import Path
from datetime import date
from datetime import timedelta
import plotly.express as px
import plotly.graph_objects as go
import hvplot.pandas

%matplotlib inline

In [11]:
# Load .env enviroment variables
load_dotenv()

True

In [12]:
quandl_api_key = os.getenv('QUANDL_API_KEY')
coin_api_key = os.getenv('COIN_API_KEY') 
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Confirm .env loaded successfully
# type(alpaca_api_key)

In [13]:
# API common parameters
coin_api_period = '1DAY'
alpaca_api_period = '1D'
start_date = pd.Timestamp('2015-01-01', tz='America/New_York').isoformat()
end_date = end_date = pd.Timestamp('2021-01-01', tz='America/New_York').isoformat()

## BTC Latest OHLCV

In [14]:
# API params
base_url = 'https://rest.coinapi.io/v1/ohlcv/'

# https://docs.coinapi.io/#list-all-periods
# 1HRS, 1DAY, 1MTH, 1YRS

# Ticker param
ticker = 'BTC'

In [15]:
# Get latest transactions for BTC
# This gets date for 100 days
coin_api_latest_btc_url = f'{base_url}{ticker}/USD/latest?apikey={coin_api_key}&period_id={coin_api_period}'
coin_api_latest_btc_url

'https://rest.coinapi.io/v1/ohlcv/BTC/USD/latest?apikey=05EDDE46-F455-4378-BE18-AFF75C11772D&period_id=1DAY'

In [16]:
# Execute get request
btc_results = requests.get(coin_api_latest_btc_url).json()

In [8]:
# What is type? 
# Answer: list of dictionaries
# type(btc_latest_results)

In [9]:
# Examine single record
# btc_results[0]

In [17]:
# Create new DataFrame from raw data (list of dicts)
btc_df = pd.DataFrame(
    btc_results,
    columns=['time_period_end','price_open','price_high','price_low','price_close','volume_traded'])

# Convert string to date
btc_df['time_period_end'] = pd.to_datetime(btc_df['time_period_end']).dt.date

# Set index to date
# btc_df.set_index('date', inplace=True)

# Drop time_period_end
# btc_df.drop(['time_period_end'], axis=1, inplace=True)

# Rename columns
btc_df.rename(columns = {'time_period_end':'date','price_open':'open','price_high':'high','price_low':'low','price_close':'close','volume_traded':'volume'}, inplace = True)

# Verify
btc_df.head()

,date,open,high,low,close,volume
0,2021-01-21,35910.20,36440.10,33374.00,35050.01,47736.371704
1,2021-01-20,36624.23,37934.20,35888.00,35917.28,34192.643115
2,2021-01-19,35820.01,37469.21,34736.46,36624.23,32241.847027
3,2021-01-18,36004.80,36873.90,33833.55,35820.00,36821.003278
4,2021-01-17,36754.60,37949.71,35342.20,36018.64,39468.975920


## BTC Historical OHLCV

In [18]:

# 'https://rest.coinapi.io/v1/ohlcv/'
# &limit={limit}&include_id={include_id}
# Get historical data for BTC

symbol_id = 'BITSTAMP_SPOT_BTC_USD'
time_start = '2015-01-01T00:00:00'
time_end = '2021-01-01T00:00:00'
coin_api_historical_btc_url = f'https://rest.coinapi.io/v1/ohlcv/{symbol_id}/history?apikey={coin_api_key}&period_id={coin_api_period}&time_start={time_start}&time_end={time_end}'
coin_api_historical_btc_url

'https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_BTC_USD/history?apikey=05EDDE46-F455-4378-BE18-AFF75C11772D&period_id=1DAY&time_start=2015-01-01T00:00:00&time_end=2021-01-01T00:00:00'

In [19]:
btc_historical_results = requests.get(coin_api_historical_btc_url).json()
btc_historical_results[0]

{'time_period_start': '2015-01-01T00:00:00.0000000Z',
 'time_period_end': '2015-01-02T00:00:00.0000000Z',
 'time_open': '2015-01-01T00:00:22.3630000Z',
 'time_close': '2015-01-01T23:59:00.9170000Z',
 'price_open': 321.0,
 'price_high': 321.0,
 'price_low': 312.6,
 'price_close': 313.81,
 'volume_traded': 3087.43655395,
 'trades_count': 2350}

In [20]:
def get_crypto_OHLCV(api_key, ticker, date_start, date_end, period) :

    # CoinAPI.io REST url for historical OHLCV
    url = f'https://rest.coinapi.io/v1/ohlcv/{ticker}/USD/history?apikey={api_key}&period_id={period}&time_start={date_start}&time_end={date_end}&limit=100000'
    
    # List of dictionary objects
    results = requests.get(url).json()

    # Init dataframe from raw results
    df = pd.DataFrame(
        results, 
        columns=['time_period_end','price_open','price_high','price_low','price_close','volume_traded'])

    # Parse date from string
    df['time_period_end'] = pd.to_datetime(df['time_period_end']).dt.date

    # Rename columns
    df.rename(
        columns = {'time_period_end':'date',
            'price_open':'open',
            'price_high':'high',
            'price_low':'low',
            'price_close':'close',
            'volume_traded':'volume'}, 
        inplace = True)

    return df

In [21]:
# Test function
btc_test_df = get_crypto_OHLCV(coin_api_key, 'BTC', '2020-01-01T00:00:00', '2021-01-01T00:00:00', '1DAY')
btc_test_df.head()

,date,open,high,low,close,volume
0,2020-01-02,7165.72,7238.14,7136.05,7174.33,7111.559960
1,2020-01-03,7174.33,7186.65,6900.00,6942.30,17021.336977
2,2020-01-04,6945.02,7402.31,6853.53,7338.91,31240.408837
3,2020-01-05,7334.45,7398.00,7256.03,7348.63,11282.295811
4,2020-01-06,7348.63,7495.00,7301.10,7355.40,12110.709259


In [15]:
# Plot daily closing
fig = px.line(
    btc_df, 
    x="date", 
    y="close", 
    title='BTC Daily Closing Prices',
    labels={'date':'Daily', 'close':'Closing Price'})

fig.show()

## SPY Historical OHLCV


In [22]:
# Init Alpaca SDK
alpaca_api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [23]:
# Format current date as ISO format
# We're acutally going to pull from three days ago to ensure closing prices available
# current_date = pd.Timestamp(date.today() - timedelta(days = 20), tz="America/New_York").isoformat()
start_date = pd.Timestamp('2015-01-01', tz="America/New_York").isoformat()
end_date = pd.Timestamp('2020-01-01', tz="America/New_York").isoformat()

# Set the tickers
tickers = ["SPY"]

# Get current closing prices for SPY and AGG
spy_raw_df = alpaca_api.get_barset(
    tickers,
    alpaca_api_period,
    start=start_date,
    end=end_date
).df

# Preview DataFrame
spy_raw_df.head()

SPY                                    
                             open    high      low   close     volume
time                                                                 
2015-01-02 00:00:00-05:00  206.38  206.88  204.180  205.41   94506045
2015-01-05 00:00:00-05:00  204.17  204.37  201.350  201.80  133100354
2015-01-06 00:00:00-05:00  202.09  202.72  198.855  199.82  166955423
2015-01-07 00:00:00-05:00  201.42  202.72  200.880  202.34  103285826
2015-01-08 00:00:00-05:00  204.00  206.16  203.990  205.92  112316394

In [24]:
spy_df = pd.DataFrame()
spy_df['open'] = spy_raw_df['SPY']['open']
spy_df['high'] = spy_raw_df['SPY']['high']
spy_df['low'] = spy_raw_df['SPY']['low']
spy_df['close'] = spy_raw_df['SPY']['close']
spy_df['volume'] = spy_raw_df['SPY']['volume']
spy_df['date'] = spy_raw_df.index.date
spy_df.reset_index(inplace=True)
spy_df = spy_df[['date','open','high','low','close','volume']]
spy_df.head()

# spy_df.index = spy_raw_df.index.date

,date,open,high,low,close,volume
0,2015-01-02,206.38,206.88,204.180,205.41,94506045
1,2015-01-05,204.17,204.37,201.350,201.80,133100354
2,2015-01-06,202.09,202.72,198.855,199.82,166955423
3,2015-01-07,201.42,202.72,200.880,202.34,103285826
4,2015-01-08,204.00,206.16,203.990,205.92,112316394


In [19]:
# spy_3_df = spy_df[['SPY']['open','high','low','close','volume']]
# spy_3_df.head()
# test = spy_raw_df[['SPY']['open']]

In [20]:
spy_df.index

RangeIndex(start=0, stop=1258, step=1)

In [21]:
# def get_stock_OHLCV(api_key, secret_key, ticker, date_start, date_end, period) :

#     # Init Alpaca SDK
#     api = tradeapi.REST(
#         api_key,
#         secret_key,
#         api_version="v2")

#     # Init start/end dates
#     start_date = pd.Timestamp(date_start, tz="America/New_York").isoformat()
#     end_date = pd.Timestamp(date_end, tz="America/New_York").isoformat()

#     raw_df = api.get_barset(
#         [ticker],
#         period,
#         start=start_date,
#         end=end_date
#     ).df

#     df = pd.DataFrame()
#     df['open'] = raw_df['SPY']['open']
#     df['high'] = raw_df['SPY']['high']
#     df['low'] = raw_df['SPY']['low']
#     df['close'] = raw_df['SPY']['close']
#     df['volume'] = raw_df['SPY']['volume']
#     df['date'] = raw_df.index.date
#     df.reset_index(inplace=True)
#     df = df[['date','open','high','low','close','volume']]

#     return df

In [22]:
# spy_ohlcv_df = get_stock_OHLCV(alpaca_api_key, alpaca_secret_key, 'SPY', '2020-01-01T00:00:00', '2021-01-01T00:00:00', '1D')
# btc_ohlcv_df.head()

In [23]:
# Get BTC OHLCV
# btc_ohlcv_df = get_crypto_OHLCV(coin_api_key, 'BTC', '2020-01-01T00:00:00', '2021-01-01T00:00:00', '1DAY')

In [24]:
# Get SPY OHLCV
# spy_ohlcv_df = get_stock_OHLCV(alpaca_api_key, alpaca_secret_key, 'SPY', '2020-01-01T00:00:00', '2021-01-01T00:00:00', '1D')

In [25]:
# btc_plot = btc_ohlcv_df.hvplot.line(
#     x='date',
#     y='close',
#     xlabel='Date',
#     ylabel='Closing Price',
#     title='BTC')

# btc_plot

In [26]:
# spy_plot = spy_ohlcv_df.hvplot.line(
#     x='date',
#     y='close',
#     xlabel='Date',
#     ylabel='Closing Price',
#     title='SPY')

# spy_plot

In [27]:
# btc_plot * spy_plot

## Refactor to data_service library and test module import

In [4]:
import data_service as ds

In [5]:
# coin_api_key = os.getenv('COIN_API_KEY') 
# alpaca_api_key = os.getenv("ALPACA_API_KEY")
# alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [25]:
# Call data_service to get SPY historical OHLCV
spy_df = ds.get_stock_OHLCV(alpaca_api_key, alpaca_secret_key, 'SPY', '2020-01-01', '2021-01-01', '1D')
spy_df.head()

,date,open,high,low,close,volume
0,2020-01-02,323.54,324.85,322.53,324.84,48217559
1,2020-01-03,321.16,323.64,321.10,322.44,58408809
2,2020-01-06,320.49,323.73,320.36,323.67,43105385
3,2020-01-07,323.02,323.54,322.24,322.76,33909984
4,2020-01-08,322.94,325.78,322.67,324.42,56581814


In [26]:
# Call data_service to get BTC historical OHLCV
btc_df = ds.get_crypto_OHLCV(coin_api_key, 'BTC', '2020-01-01T00:00:00', '2021-01-01T00:00:00', '1DAY')
btc_df.head()


,date,open,high,low,close,volume
0,2020-01-02,7165.72,7238.14,7136.05,7174.33,7111.559960
1,2020-01-03,7174.33,7186.65,6900.00,6942.30,17021.336977
2,2020-01-04,6945.02,7402.31,6853.53,7338.91,31240.408837
3,2020-01-05,7334.45,7398.00,7256.03,7348.63,11282.295811
4,2020-01-06,7348.63,7495.00,7301.10,7355.40,12110.709259
